In [9]:
include("StackyFan.jl")

interiorPoints (generic function with 1 method)

In [10]:
#inputs are a stacky fan and a vector of booleans representing the distinguished structure.
function BerghA(X::StackyFan,D::Array{Int64,1})
    rayMatrix=Array(X.fan.RAYS)
    coneList=convertIncidenceMatrix(X.fan.MAXIMAL_CONES)
    dim=size(rayMatrix,2)
    l=size(rayMatrix,1)
    #check if the vector D has length equal to the number of rays in F
    if l != size(D,1)
        error("length of vector representing distinguished structure does not agree with number of rays in stacky fan.")
    end
    
    #A0
    i=0
    while(true)
        #A1
        #Note: cones in S are 1-indexed.
        S=filter(cone->distinguishedAndMultiplicity(cone,rayMatrix,D),coneList)
        if S==[]
            break
        end
        
        #A2 - find extremal cones
        Smax=extremalCones(S,rayMatrix,D)
        return Smax
        
    end
    return X
end

BerghA (generic function with 1 method)

In [11]:
F=StackyFan([1 0; 1 2; 1 -1],[[0,1],[0,2]],[1,1,1])

StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x00007ffc7d4357d0), [1, 1, 1], Dict("1,2" => 1, "1,-1" => 1, "1,0" => 1))

In [12]:
BerghA(F,[0,0,1])

StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x00007ffc7d4357d0), [1, 1, 1], Dict("1,2" => 1, "1,-1" => 1, "1,0" => 1))

In [14]:
lattice = X.HASSE_DIAGRAM
faces = @Polymake.convert_to Array{Set{Int}} lattice.FACES

LoadError: type StackyFan has no field HASSE_DIAGRAM

In [46]:
getCones(X)

13-element Array{Any,1}:
 [0, 1, 2, 3]
 [0, 2, 4]
 [0, 1]
 [0, 2]
 [2, 3]
 [1, 3]
 [0, 4]
 [2, 4]
 [0]
 [1]
 [2]
 [3]
 [4]